In [1]:
import torch
import torch.nn as nn


In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [13]:
batchs=torch.stack((inputs,inputs),dim=0)
batchs.shape

torch.Size([2, 6, 3])

In [14]:
class casualattention(nn.Module):
    def __init__(self,din,dout,biasbool,batchsize,dropoutsize):
        super().__init__()
        self.W_query=nn.Linear(din,dout,bias=biasbool)
        self.W_keys=nn.Linear(din,dout,bias=biasbool)
        self.W_values=nn.Linear(din,dout,bias=biasbool)
        self.dropout=nn.Dropout(dropoutsize)
        self.register_buffer('mask', torch.triu(torch.ones(batchsize, batchsize), diagonal=1))

    def forward(self,x):
        b, num_tokens, d_in = x.shape
        query= self.W_query (x)
        keys= self.W_keys (x)
        values=self.W_values (x)

        scores=query @ keys.transpose(1,2)
        scores.masked_fill_(  
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights=torch.softmax(scores/(keys.shape[-1])**0.5,dim=-1)
        attn_weights = self.dropout(attn_weights) 

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)

context_length = batchs.shape[1]
ca = casualattention(3, 2,False, context_length, 0.0)

context_vecs = ca(batchs)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

        



tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])
